In [1]:
import os
import json 

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token

# Load Model and Tokenizer

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import LlamaTokenizerFast


accelerator = Accelerator()
device = accelerator.device

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B"  # non-instruct version

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    # token=access_token,
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_id, token=access_token)

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.41.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:49<00:00, 12.49s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load data and define dataloaders

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader

imdb = load_dataset("imdb")
train_ds = imdb['train']


def collate_fn(examples):
    max_len = 512
    texts = [example['text'] for example in examples]
    texts = [text if len(text) <= max_len else text[:max_len] for text in texts]
    labels = [example['label'] for example in examples]
    messages_lambda = lambda texts: [
        {"role": "system", "content": "You are a chatbot for sentiment analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
        # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
        {"role": "user", "content": texts},
    ]
    messages = list(map(messages_lambda, texts))

    messages_with_template_applied = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    batch = tokenizer(
                messages_with_template_applied,
                add_special_tokens=False,
                padding=True,
                return_tensors="pt",
                )
    
    # find the template boundaries
    text_lens = [len(tokenizer.encode(text)) for text in texts]
    text_lens_tensor = torch.tensor(text_lens, dtype=torch.long)
    

    def apply_mask(mask_tensor, text_lens_tensor):
        batch_size, seq_len = mask_tensor.shape
        for i in range(batch_size):
            text_len = text_lens_tensor[i].item()
            mask_tensor[i, -text_len-5:-5] = 0
        return 1- mask_tensor

    mask_tensor = apply_mask(torch.ones_like(batch['input_ids']), text_lens_tensor)

    batch['context_mask'] = mask_tensor

    
    return batch


# Define batch size here!
batch_size = 16
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


# Train model

In [5]:
from llmexp.model2 import MaskGeneratingModel
mask_gen_model = MaskGeneratingModel(hidden_size=4096, mlp_hidden_dim=1024, mlp_bottleneck_dim=768, mlp_num_blocks=2)
mask_gen_model.to(device)


from tqdm import tqdm

# Set pad_token_id if it is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pad_token_id = tokenizer.pad_token_id

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

mask_gen_model.load_state_dict(torch.load("saved_model/mask_gen_model_1_3000.pth"))

<All keys matched successfully>

# Demo examples

In [26]:
import numpy as np
idx = 0
from captum.attr import visualization as viz
import torch.nn.functional as F

# tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film. The acting was great, and the story was amazing. I would recommend this movie to everyone."
# texts = "I don't like this movie. The acting was terrible, and the story was boring. I would not recommend this movie to anyone."
test_text = [{"text": texts, "label": None}]
test_inputs = collate_fn(test_text).to(device)

# test_inputs = next(iter(train_dataloader)).to(device)
tokens = tokenizer.convert_ids_to_tokens(test_inputs['input_ids'][0])

# generate the answer for the test inputs
gen_outputs = model.generate(
            input_ids=test_inputs['input_ids'],
            attention_mask=test_inputs['attention_mask'],
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
input_ids = test_inputs['input_ids']
attention_mask = test_inputs['attention_mask']
gen_tokens = gen_outputs.sequences
pad_length = gen_tokens.size(1) - input_ids.size(1)
# get the attention mask for the generated tokens, and also mask the padding tokens
gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
# (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
unpaded_token_mask = (gen_tokens != pad_token_id).long()
unpaded_token_mask[:, :-pad_length] = 1
gen_attention_mask = gen_attention_mask * unpaded_token_mask

with torch.no_grad():
    prompt_outputs = model(input_ids=test_inputs['input_ids'], attention_mask=test_inputs['attention_mask'], output_hidden_states=True, return_dict=True)
    last_hidden_state = prompt_outputs.hidden_states[-1].float()
    mask_logits = mask_gen_model(last_hidden_state)

# Function to clean tokens
def clean_token(token):
    # Remove special characters like "Ġ" or "Ċ"
    return token.replace("Ġ", "").replace("Ċ", "").replace("Ġ", "").replace("Ċ", "").replace("Ġ", "")

# Apply cleaning to each token
tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
tokens = [clean_token(token) for token in tokens]

def normalize_except_zeros(array):
    # Create a mask to identify non-zero elements
    mask = array != 0
    
    # Extract non-zero elements
    non_zero_elements = array[mask]
    
    # Normalize non-zero elements
    min_val = np.min(non_zero_elements)
    max_val = np.max(non_zero_elements)
    normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
    
    # Create a copy of the original array to preserve zero values
    normalized_array = np.copy(array)
    
    # Assign normalized values back to the corresponding positions
    normalized_array[mask] = normalized_non_zero_elements
    
    return normalized_array

expl = (torch.sigmoid(mask_logits) * test_inputs['context_mask'])[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)
expl_raw = expl.detach().cpu().numpy()
# expl = (expl - 0) / (expl_raw.max(axis=-1) - 0)


expl = normalize_except_zeros(expl_raw)


# expl = response_mask[idx]


vis_data_records = [viz.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
viz.visualize_text(vis_data_records)

In [23]:
expl_raw

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.46262485, 0.4555001 , 0.48219326, 0.46485275,
       0.47883537, 0.49213785, 0.46462926, 0.46440512, 0.46345183,
       0.47215888, 0.44916716, 0.45338422, 0.45447388, 0.4626619 ,
       0.47822577, 0.46337965, 0.46034363, 0.47810313, 0.46203044,
       0.46400258, 0.47872314, 0.47321   , 0.4623515 , 0.46870413,
       0.4636747 , 0.46463996, 0.4546705 , 0.45732358, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ], dtype=float32)

In [16]:
torch.sigmoid(mask_logits)

tensor([[0.5215, 0.4805, 0.4850, 0.4635, 0.4868, 0.4523, 0.4755, 0.4939, 0.4638,
         0.4888, 0.4758, 0.4621, 0.4598, 0.4727, 0.4652, 0.4721, 0.4815, 0.4706,
         0.4678, 0.4783, 0.4842, 0.4715, 0.4804, 0.4563, 0.4677, 0.4768, 0.4657,
         0.4818, 0.4740, 0.4652, 0.4639, 0.4599, 0.4896, 0.4829, 0.4653, 0.4695,
         0.4612, 0.4782, 0.4779, 0.4698, 0.4766, 0.4468, 0.4539, 0.4642, 0.4818,
         0.4741, 0.4616, 0.4724]], device='cuda:0')

In [9]:
expl = (torch.sigmoid(mask_logits) * test_inputs['context_mask'])[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)
expl.detach().cpu().numpy()

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.463446  , 0.4629632 , 0.4716442 , 0.4840185 ,
       0.46899056, 0.4736643 , 0.4788765 , 0.46411845, 0.45112532,
       0.464376  , 0.45868424, 0.4531282 , 0.46908346, 0.47324648,
       0.4640558 , 0.44945025, 0.45505166, 0.44949603, 0.43960032,
       0.46445277, 0.46677357, 0.45706436, 0.4594847 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ], dtype=float32)

In [73]:
expl = expl.detach().cpu().numpy()
non_zero_elements = expl[expl!=0]
# Normalize non-zero elements
min_val = np.min(non_zero_elements)
max_val = np.max(non_zero_elements)
normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
normalized_non_zero_elements

array([1.        , 0.8167707 , 0.67404723, 0.6338016 , 0.7690665 ,
       0.        ], dtype=float32)

In [72]:
non_zero_elements

tensor([0.3475, 0.3165, 0.2923, 0.2855, 0.3084, 0.1781], device='cuda:0')

In [68]:
expl = normalize_except_zeros(expl.detach().cpu().numpy())
expl

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.8167707 , 0.67404723,
       0.6338016 , 0.7690665 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ], dtype=float32)

In [161]:
print(correct_logprob[0,:10])
print(correct_logprob_gt[0,:10])

tensor([-11.8911, -12.1255, -11.0005, -10.5004,  -9.8753,  -9.7502, -10.0002,
        -10.2501, -10.1251, -10.2501], device='cuda:0')
tensor([-7.7515, -7.1261, -7.5007, -7.1260, -6.3769, -5.8780, -6.2521, -6.2521,
        -6.0027, -6.1274], device='cuda:0')


In [104]:
gen_tokens

tensor([[128009, 128009, 128009,  ...,   4632,     13, 128009],
        [128000, 128006,   9125,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        ...,
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009],
        [128009, 128009, 128009,  ..., 128009, 128009, 128009]],
       device='cuda:0')

# Demo examples 

In [146]:
print(expl)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [66]:

print(response_mask[idx])
# print("context_mask\n", context_mask[idx])
# print("user_input_mask\n",user_input_mask[idx].long())
print("masked_attention_mask\n",masked_attention_mask[idx].long())
# print("response_mask\n", response_mask[idx].long())
print("gen_attention_mask\n", gen_attention_mask[idx].long())


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
masked_attention_m

In [72]:
tokenizer.decode(perturbed_input_ids[idx] * masked_attention_mask[idx].long())

'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, NEGATIVE OR NEUTURAL with a brief explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThis!!!!!!!!! any!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! by!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! bag!!!!!!! the!!!!!!!!!!!!!!!!!!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNEGATIVE\n\nThe reviewer expresses their disappointment and frustration with the movie, calling it a "classic" only because of its sentiment rather than its quality. They criticize the plot as "unbelievable", "unrealistic", and "ludicrous", and highlight specific scenes as unrealistic, such as a regiment marching into an ambush. The tone is overwhelmingly negative.!!!!!!!!!!!!!!!!!!!!!'

In [77]:
torch.sigmoid(mask_logits)[idx]

tensor([7.3034e-02, 2.3672e-01, 1.6484e-01, 5.7779e-02, 4.5203e-02, 3.7355e-02,
        5.0271e-02, 6.9986e-02, 6.1506e-02, 4.2663e-02, 4.4438e-02, 7.5177e-02,
        3.5435e-02, 5.5367e-02, 3.7714e-02, 3.1168e-02, 4.1736e-02, 4.3801e-02,
        3.1036e-02, 3.4357e-02, 5.7400e-02, 3.6266e-02, 4.7387e-02, 9.0895e-02,
        4.7148e-02, 5.1576e-02, 5.1955e-02, 4.8488e-02, 6.0373e-02, 6.3567e-02,
        6.8870e-02, 8.2406e-02, 6.4658e-02, 5.7909e-02, 6.6989e-02, 5.4006e-02,
        5.4770e-02, 1.3873e-01, 7.5017e-01, 1.8232e-01, 2.9083e-02, 7.7919e-02,
        2.8039e-01, 1.2871e-01, 6.7355e-01, 5.4293e-01, 5.2408e-02, 1.3911e-01,
        7.0787e-01, 3.0805e-01, 3.1633e-01, 1.9815e-02, 7.9728e-02, 1.2592e-01,
        2.8685e-01, 7.6748e-01, 3.5774e-01, 7.0721e-01, 8.3341e-01, 1.0890e-01,
        4.6710e-01, 4.1535e-02, 1.5163e-02, 9.8794e-02, 1.5306e-01, 4.1573e-02,
        3.1574e-01, 7.2187e-01, 3.3909e-02, 1.4620e-01, 2.2021e-01, 2.9110e-01,
        2.8944e-02, 2.2404e-01, 3.1191e-

In [87]:
from captum.attr import visualization as viz

tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# Function to clean tokens
def clean_token(token):
    # Remove special characters like "Ġ" or "Ċ"
    return token.replace("Ġ", "").replace("Ċ", "").replace("Ġ", "").replace("Ċ", "").replace("Ġ", "")

# Apply cleaning to each token
tokens = [clean_token(token) for token in tokens]

expl = (torch.sigmoid(mask_logits) * context_mask)[idx]
expl = F.pad(expl, (0, len(tokens) - expl.size(0)), mode='constant', value=0)


vis_data_records = [viz.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
viz.visualize_text(vis_data_records)

AttributeError: 'LlamaForCausalLM' object has no attribute 'tokenizer'